In [1]:
import pandas as pd
import numpy as np
import re
import os
import time
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import nltk
import faiss

In [15]:
!pip install faiss-cpu

In [2]:
file_path = "Corpus_completo.xlsx"
corpus_completo = pd.read_excel(file_path)


In [3]:
# Formatear fechas

meses = {
    'enero': '01',
    'febrero': '02',
    'marzo': '03',
    'abril': '04',
    'mayo': '05',
    'junio': '06',
    'julio': '07',
    'agosto': '08',
    'septiembre': '09',
    'octubre': '10',
    'noviembre': '11',
    'diciembre': '12'
}

def convertir_fecha(fecha_str):
    partes = fecha_str.split(' de ')
    dia = partes[0].zfill(2)  # Asegura 2 dígitos para días 1-9
    mes = meses[partes[1]]
    año = partes[2]
    return f"{dia}/{mes}/{año}"  # Formato DD/MM/AAAA

corpus_completo['Fecha_formateada'] = corpus_completo['Fecha'].apply(convertir_fecha)
corpus_completo['Fecha_formateada'] = pd.to_datetime(corpus_completo['Fecha_formateada'], format='%d/%m/%Y', errors='coerce')

In [4]:
corpus_completo

,Diario,Autor,Fecha,Título,Texto,Vínculo,ID,Fecha_formateada
0,El Espectador,Gonzalo Hernández,1 de enero de 2018,Fajardo: para nada tibio,"La Coalición Colombia Partido Alianza Verde, ...",https://web.archive.org/web/20180102104221/htt...,1,2018-01-01
1,El Espectador,Eduardo Barajas Sandoval,1 de enero de 2018,Macedonia de Norte,Las interpretaciones de la historia sirven com...,https://web.archive.org/web/20180102104221/htt...,2,2018-01-01
2,El Espectador,Daniel Emilio Rojas Castro,1 de enero de 2018,El nacionalismo según Vargas Llosa,La semana pasada Mario Vargas Llosa publicó un...,https://web.archive.org/web/20180102104221/htt...,3,2018-01-01
3,El Espectador,Reinaldo Spitaletta,1 de enero de 2018,"Tiempo sagrado, tiempo profano","Pudiera decirse, sin ser una verdad absoluta, ...",https://web.archive.org/web/20180102104221/htt...,4,2018-01-01
4,El Espectador,Aura Lucía Mera,1 de enero de 2018,La rebelión de los bueyes,Lo mejor del encierro de Las Ventas fueron los...,https://web.archive.org/web/20180102104221/htt...,5,2018-01-01
...,...,...,...,...,...,...,...,...
13335,Semana,Esteban Piedrahita,25 de diciembre de 2018,Capitalismo a la escandinava,Los países nórdicos coquetearon con el sociali...,https://web.archive.org/web/20181227221015/htt...,13336,2018-12-25
13336,Semana,Germán Manga,26 de diciembre de 2018,"Ahora, a derrotar a los Guachos del Amazonas",Solo una acción militar sostenida y de gran es...,https://web.archive.org/web/20181227220937/htt...,13337,2018-12-26
13337,Semana,Ariel Ávila,26 de diciembre de 2018,Porte de armas y Estado de derecho,El debate del porte de armas es más complejo d...,https://web.archive.org/web/20181227220928/htt...,13338,2018-12-26
13338,Semana,Lucas Pombo,27 de diciembre de 2018,Quién es quién en el Congreso,"En estos primeros meses de actividad, algunos ...",https://web.archive.org/web/20181227220919/htt...,13339,2018-12-27


In [5]:
# Generar embeddings de textos
modelo_embedding = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')


In [6]:
# elegir  3000 columnas aleatoriamente
corpus_filtro = corpus_completo.sample(n=3000, random_state=42)
textos = corpus_filtro['Texto'].astype(str).tolist()  # Convertir a lista de strings
#textos = corpus_completo['Texto'][:3000].astype(str).tolist()  
embeddings = modelo_embedding.encode(textos, convert_to_tensor=False).astype('float32')


In [7]:
# Crear índice FAISS
indice = faiss.IndexFlatL2(embeddings.shape[1])
indice.add(embeddings)

In [26]:
# Buscar texto similar a una consulta
consulta = ["literatura"]
embedding_consulta = modelo_embedding.encode(consulta, convert_to_tensor=False).astype('float32')
distancias, indices = indice.search(embedding_consulta, k=20)

In [22]:
distancias

array([[4.220034 , 4.2220078, 4.293207 , 4.4357433, 4.45562  , 4.505492 ,
        4.5993633, 4.6275597, 4.641143 , 4.764921 , 4.8358126, 4.8580513,
        4.873453 , 4.9067044, 4.9699717, 4.973748 , 5.047427 , 5.067581 ,
        5.105868 , 5.111073 ]], dtype=float32)

In [27]:
indices[0]  # Índices de los textos más similares

array([2174, 1217, 2619,  387, 1037, 1767, 2642,  154, 2900,  223, 2668,
       1819,  873, 2916,  352, 2456, 1837, 1335, 1671, 1456], dtype=int64)

In [30]:
# Filtrar la fila
print(corpus_filtro.iloc[2642])

Diario                                                  El Espectador
Autor                                            Esteban Carlos Mejía
Fecha                                             28 de junio de 2019
Título                                     Más de 178 muertos por día
Texto               Me encanta leer historia patria, pero en novel...
Vínculo             https://web.archive.org/web/20190629223610/htt...
ID                                                               4768
Fecha_formateada                                  2019-06-28 00:00:00
Name: 4767, dtype: object


In [25]:
#Resultados
print("Textos más similares:")
for idx in indices[0]:
    print(f"Índice: {idx}")
    print("Texto: " + textos[idx])
    print("---------------------------------------")


Textos más similares:
Índice: 2174
Texto: En 1981 el filósofo australiano Peter Singer (quien ocasionalmente escribe para este periódico) publicó el libro El círculo en expansión; ética, evolución y progreso moral. En un comienzo llamó poco la atención, pero con el tiempo se convirtió en lectura obligada para los estudiosos formales e informales de la ética y la moral.

Se basa en la sociobiología, término acuñado unos años antes por el biólogo Edward Wilson, estudioso de las hormigas y otros insectos sociales. Hasta hoy, Wilson es vetado en algunas universidades por grupos de activistas que se autocalifican de progresistas y desconocen los orígenes biológicos de la moral, negándose a aceptar nuestra evidente animalidad.

Singer conjuga los análisis evolutivos de Wilson con concepciones sobre moral y justicia, de Rawls especialmente. Parte del desarrollo de las normas sociales en primates, y de evidencias antropológicas sobre sociedades de cazadores-recolectores, y llega a hoy. Sus con